In [1]:
import pandas as pd
from selenium.webdriver.common.by import By

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import datetime
import time
import os
import random
import pickle
import undetected_chromedriver as uc

In [ ]:
# 1. Hàm lấy danh sách URL
def get_url(places):
    url = []
    date = []
    #today = datetime.datetime.today().date()
    today = datetime.date(2026, 3, 1)
    date.append(today)
    for i in range(1, 20):  # Điều chỉnh số ngày cào tại đây
        date.append(today + datetime.timedelta(days=i))
    
    for place in places:
        for day in date:
            url.append(f"https://www.traveloka.com/vi-vn/flight/fullsearch?ap={place}&dt={day.strftime('%d-%m-%Y')}.NA&ps=1.0.0&sc=ECONOMY")
            
    return url

# Cell 2: Hàm cào dữ liệu từ Traveloka
# 1. Hàm giả lập hành vi người dùng
def simulate_human_behavior(driver):
    action = ActionChains(driver)
    
    elements = driver.find_elements(By.XPATH, "//button | //div[not(ancestor::a) and not(@onclick)]")  
    if elements:
        element = random.choice(elements)
        try:
            action.move_to_element(element).click().perform()
            print("Nhấp vào phần tử không điều hướng")
        except:
            print("Không thể nhấp vào phần tử")

    # Cuộn trang
    driver.execute_script("window.scrollBy(0, 1000);")
    time.sleep(random.uniform(2, 5))


def crawl_planetrip(url_list):
    options = Options()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36")
    driver = uc.Chrome(options=options)

    print("🚀 ChromeDriver đã khởi chạy! Đang tải cookies...")
    driver.get("https://www.traveloka.com")
    #load_cookies(driver)
    time.sleep(5)
    driver.refresh()

    all_data = []
    for url in url_list:
        driver.get(url)
        time.sleep(random.randint(10, 20))  # Giảm nguy cơ bị chặn
        simulate_human_behavior(driver)  # Giả lập hành vi người dùng
        # Lấy ngày từ URL
        start_day = url.split("&dt=")[1].split(".")[0]
        flights = driver.find_elements(By.CSS_SELECTOR, "div.css-1dbjc4n.r-1x4r79x")
        print(f"Tìm thấy {len(flights)} chuyến bay")
        for flight in flights:
            try:
                airline = flight.find_element(By.CSS_SELECTOR, "div.css-1dbjc4n.r-1habvwh.r-18u37iz.r-1ssbvtb").text
            except:
                airline = "N/A"
            
            try:
                price = flight.find_element(By.CSS_SELECTOR, "div.css-1dbjc4n.r-obd0qt.r-eqz5dr.r-9aw3ui.r-knv0ih.r-ggk5by").text
            except:
                price = "0"
            try:
                start_time = flight.find_element(By.CSS_SELECTOR, "div.css-1dbjc4n.r-1habvwh.r-eqz5dr.r-9aw3ui.r-knv0ih").text
            except:
                start_time = "N/A"
            try:
                total_time = flight.find_element(By.CSS_SELECTOR, "div.css-1dbjc4n.r-1awozwy.r-eqz5dr.r-knv0ih.r-bnwqim").text
            except:
                total_time = "N/A"
            try:
                end_time = flight.find_element(By.CSS_SELECTOR, "div.css-1dbjc4n.r-obd0qt.r-eqz5dr.r-9aw3ui.r-knv0ih").text
            except:
                end_time = "N/A"
            
            data = {"Airline": airline, "Price": price, "Start_time": start_time, "Total_time": total_time, "End_time": end_time, "Start_Day": start_day}
            #print(len(data))
            all_data.append(data)

    print("💾 Lưu cookies để sử dụng lần sau")
    #save_cookies(driver)
    driver.quit()
    return pd.DataFrame(all_data)

# 3. Hàm tiền xử lý dữ liệu
def preprocessing_data(df):
    df["Price"] = pd.to_numeric(df["Price"].str.replace("\D", "", regex=True), errors="coerce")
    df["Price"] = df["Price"].fillna(0).astype("int64")  # Đổi thành int64 nếu cần
    return df

In [21]:
# 4. Thực thi các hàm
places = ["DAD.DLI", "DAD.PQC", "DAD.VDO"]  # Định nghĩa danh sách các tuyến bay
# places = ["DAD.DLI",]
urls = get_url(places)

df_raw = crawl_planetrip(urls)
df_clean = preprocessing_data(df_raw)

🚀 ChromeDriver đã khởi chạy! Đang tải cookies...
Nhấp vào phần tử không điều hướng
Tìm thấy 14 chuyến bay
Nhấp vào phần tử không điều hướng
Tìm thấy 23 chuyến bay
Nhấp vào phần tử không điều hướng
Tìm thấy 142 chuyến bay
Nhấp vào phần tử không điều hướng
Tìm thấy 44 chuyến bay
Nhấp vào phần tử không điều hướng
Tìm thấy 58 chuyến bay
Nhấp vào phần tử không điều hướng
Tìm thấy 72 chuyến bay
Nhấp vào phần tử không điều hướng
Tìm thấy 72 chuyến bay
Nhấp vào phần tử không điều hướng
Tìm thấy 72 chuyến bay
Nhấp vào phần tử không điều hướng
Tìm thấy 144 chuyến bay
Nhấp vào phần tử không điều hướng
Tìm thấy 51 chuyến bay
Nhấp vào phần tử không điều hướng
Tìm thấy 70 chuyến bay
Nhấp vào phần tử không điều hướng
Tìm thấy 72 chuyến bay
Nhấp vào phần tử không điều hướng
Tìm thấy 14 chuyến bay
Nhấp vào phần tử không điều hướng
Tìm thấy 72 chuyến bay
Nhấp vào phần tử không điều hướng
Tìm thấy 16 chuyến bay
Nhấp vào phần tử không điều hướng
Tìm thấy 9 chuyến bay
Không thể nhấp vào phần tử
Tìm thấy 9 

In [22]:
df_clean = df_clean.drop_duplicates()


In [23]:
df_clean

,Airline,Price,Start_time,Total_time,End_time,Start_Day
0,Vietnam Airlines,2588528,06:05\nDAD,1h 10m\nBay thẳng,07:15\nDLI,01-03-2026
5,N/A,0,N/A,N/A,N/A,01-03-2026
7,"VietJet Air, Vietnam Airlines",5174479,07:35\nDAD,6h 10m\n1 điểm dừng\nTự trung chuyển,13:45\nDLI,01-03-2026
14,Vietnam Airlines,2588528,06:05\nDAD,1h 10m\nBay thẳng,07:15\nDLI,02-03-2026
19,N/A,0,N/A,N/A,N/A,02-03-2026
...,...,...,...,...,...,...
1733,,0,,,,18-03-2026
1738,N/A,0,N/A,N/A,N/A,18-03-2026
1877,,0,,,,20-03-2026
1882,N/A,0,N/A,N/A,N/A,20-03-2026


In [24]:
# 5. Xuất dữ liệu ra file CSV
df_clean.to_csv("traveloka_flights_1_T3_20_T3_2026.csv", index=False)
print("Dữ liệu đã được lưu!")

Dữ liệu đã được lưu!


In [5]:
import os
import pandas as pd

# Thư mục chứa các file CSV
folder_path = r"D:\ADMIN\Desktop\crawlingdata"

# Lấy danh sách file CSV có chứa chữ "traveloka"
csv_files = [f for f in os.listdir(folder_path) if "tour" in f.lower() and f.endswith(".csv")]

# Danh sách để lưu DataFrame của từng file
dfs = []

# Duyệt qua từng file và đọc dữ liệu
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    
    # In tên file và số dòng để debug
    print(f"File: {file}, Số dòng: {df.shape[0]}")
    
    dfs.append(df)

# Gộp tất cả file thành một DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

combined_df = combined_df.drop_duplicates()
display(combined_df)
# Xuất ra file mới (tuỳ chọn)
combined_df.to_csv("tour_traveloka_total.csv", index=False)


print(f"Tổng số dòng sau khi gộp: {combined_df.shape[0]}")


File: tour_DNPQ .csv, Số dòng: 26
File: tour_HL.csv, Số dòng: 118
File: tour_traveloka_total.csv, Số dòng: 141


,tour_name,location,price,duration,rating
0,Đi bộ dưới biển Seawalker và khám phá 3 đảo Ph...,"\nNguyễn Văn cừ, tổ 4, kp1, Phú Quốc, Kiên Gia...",1.990.000 VND,9 Hours,"9,4"
1,"Tour khám phá 4 đảo, trải nghiệm cáp treo Hòn ...","143 Trần Hưng Đạo, KP 7, TT Dương Đông, H.Phú ...",1.364.000 VND,8 Hours 30 Minutes,"8,5"
2,"Đi bộ dưới biển Seawalker, khám phá 4 đảo và c...","Tp. Phú Quốc, Kiên Giang, Vietnam",2.850.000 VND,10 Hours,"9,6"
3,Tour khám phá 3 đảo bằng tàu tại Phú Quốc - 1 ...,"Gam Ghi island, Hòn Thơm, Phu Quoc, Kien Giang...",605.000 VND,9 Hours,"8,7"
4,"Khám phá 2 đảo, Sun World Hòn Thơm, Bữa trưa b...","Phú Quốc, Phu Quoc, Kien Giang, Vietnam",1.404.000 VND\n,9 Hours 30 Minutes,"10,0"
...,...,...,...,...,...
138,[Route 2] 2D1N Halong Bay Cruise Tour by Mila ...,"Tuan Chau Marina: Tuan Chau, Ha Long, Quang Ninh",3.787.161 VND,Không tìm thấy,-
140,Ha Long Adventure: V'Spirit Cruise in Lan Ha B...,"Hạ Long, Quảng Ninh, Vietnam",1.000.000 VND,Thời gian tour | 14 Hours,"9,6"
141,Dining Experience by 5-star Luna Ha Long Cruise,"Hạ Long Bay, Thành phố Hạ Long, Quảng Ninh, Vi...",748.125 VND,Thời gian tour | 12 Hours,"9,5"
142,[New Route] 2D1N Lan Ha Bay by Le Journey Eleg...,"Hạ Long Bay, Thành phố Hạ Long, Quảng Ninh, Vi...",2.312.500 VND,Thời gian tour | 2 Days,"10,0"


Tổng số dòng sau khi gộp: 141
